<a href="https://colab.research.google.com/github/tmiqueluzzi/florianopolis_x_mapbiomas/blob/main/ReprojetaRasters.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Read Me

- Código copiado de um notebook no Jupyter
- Os arquivos utilizados no código, os arquivos de entrada assim como os arquivos de saída, estão na mesma pasta do notebook

## Explicação

- Código que pega um bloco de rasters (aqui, com sistemas de coordenadas 4326 WGS 84) e os reprojeta para um CRS desejado (no caso, precisamos de um geográfico para o cálculo da área)

# Código

In [ ]:
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling

dst_crs = "EPSG:31982" #CRS destino
resolucao = 30

for ano in range(1985, 2025):
    raster_path = f"florianopolis_{ano}_v1.tif" #os rasters com buffer estão assim
    output_tif = f"{ano}_31982.tif"

    with rasterio.open(raster_path) as src:
        transform, width, height = calculate_default_transform(
            src.crs,
            dst_crs,
            src.width,
            src.height,
            *src.bounds,
            resolution=resolucao
        )

        meta = src.meta.copy()
        meta.update({
            "driver": "GTiff",
            "crs": dst_crs,
            "transform": transform,
            "width": width,
            "height": height,
            "nodata": src.nodata
        })

        with rasterio.open(output_tif, "w", **meta) as dst:
            for i in range(1, src.count + 1):
                reproject(
                    source=rasterio.band(src, i),
                    destination=rasterio.band(dst, i),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=transform,
                    dst_crs=dst_crs,
                    resampling=Resampling.nearest
                )